In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import warnings

warnings.filterwarnings(category=FutureWarning, action="ignore")

In [29]:
df = pd.read_csv("ffiles/train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [30]:
df.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"], inplace=True)

In [31]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [32]:
# impute in age
# impute  most frequent in embarked
# ohe in sex,Embarked

In [33]:
xtrain, xtest, ytrain, ytest = train_test_split(
    df.drop(columns=["Survived"]), df["Survived"], test_size=0.2, random_state=42
)
print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)

(712, 7) (179, 7) (712,) (179,)


In [34]:
xtrain.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


# Imputation transformer


In [ ]:
trf1 = ColumnTransformer(
    [
        ("impute_age", SimpleImputer(strategy="mean"), [2]),
        ("impute_embarked", SimpleImputer(strategy="most_frequent"), [6]),
    ],
    remainder="passthrough",
)

In [62]:
xtrainn1=trf1.fit_transform(xtrain)
xtrainn1[0]

array([45.5, 'S', 1, 'male', 0, 0, 28.5], dtype=object)

# order changed after imputation

## One hot encoding


In [63]:
trf2 = ColumnTransformer(
    [
        (
            "ohe_sex_n_embarked",
            OneHotEncoder(sparse_output=False, handle_unknown="ignore",drop="first"),
            [1, 3],
        )
    ],
    remainder="passthrough",
)

In [64]:
trf2

ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe_sex_n_embarked',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore',
                                               sparse_output=False),
                                 [1, 3])])

In [65]:
xtrain

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


In [66]:
trf3 = DecisionTreeClassifier()

# Create pipeline


In [67]:
pipe = Pipeline([("trf1", trf1), ("trf2", trf2), ("trf3", trf3)])

In [68]:
from sklearn import set_config

set_config(display="diagram")

In [69]:
pipe.fit(xtrain, ytrain)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_n_embarked',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 3])])),
                ('trf3', DecisionTreeClassifier())])

# view stastics of steps 

In [80]:
pipe.named_steps["trf2"]

ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe_sex_n_embarked',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore',
                                               sparse_output=False),
                                 [1, 3])])

In [81]:
pipe.named_steps["trf1"].named_transformers_["impute_age"].statistics_

array([29.49884615])

In [82]:
ypred = pipe.predict(xtest)

In [83]:
ypred

array([0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 1])

In [85]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print("Accuracy:", accuracy_score(ytest, ypred))
print("Confusion Matrix:\n", confusion_matrix(ytest, ypred))
print("Classification Report:\n", classification_report(ytest, ypred))

Accuracy: 0.7877094972067039
Confusion Matrix:
 [[88 17]
 [21 53]]
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.84      0.82       105
           1       0.76      0.72      0.74        74

    accuracy                           0.79       179
   macro avg       0.78      0.78      0.78       179
weighted avg       0.79      0.79      0.79       179



# Dump model 

In [87]:
import pickle
pickle.dump(pipe, open("model/modeltitanic_model.pkl", "wb"))